In [0]:
# Load the dataset from DBFS (Databricks Filesystem)
file_path = "/databricks-datasets/samples/population-vs-price/data_geo.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- 2014 rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: integer (nullable = true)
 |-- 2015 median sales price: double (nullable = true)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  30101

In [0]:
# Select relevant columns for analysis
df_selected = df.select(
    "City", "State", "State Code", 
    "2014 Population estimate", "2015 median sales price"
)
df_selected.show(5)

+-------------+-------+----------+------------------------+-----------------------+
|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+-------------+-------+----------+------------------------+-----------------------+
|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|       Mobile|Alabama|        AL|                  194675|                  122.5|
|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



Top 10 Most Popular Cities

In [0]:
display(df.orderBy("2014 rank").limit(10))

2014 rank,City,State,State Code,2014 Population estimate,2015 median sales price
1,New York[6],New York,NY,8491079,388.6
2,Los Angeles,California,CA,3928864,434.7
3,Chicago,Illinois,IL,2722389,192.5
4,Houston[7],Texas,TX,2239558,200.3
5,Philadelphia[8],Pennsylvania,PA,1560297,204.9
6,Phoenix,Arizona,AZ,1537058,206.1
7,San Antonio,Texas,TX,1436697,184.7
8,San Diego,California,CA,1381069,510.3
9,Dallas,Texas,TX,1281047,192.5
10,San Jose,California,CA,1015785,900.0


Databricks visualization. Run in Databricks to view.

Average median sales price by State

In [0]:
from pyspark.sql.functions import avg
df.groupBy("State").agg(avg("2015 median sales price").alias("avg")).orderBy("avg",ascending=False).show()

+--------------------+------------------+
|               State|               avg|
+--------------------+------------------+
|             Hawai'i|             699.3|
|          California| 547.9714285714286|
|District of Columbia|             367.8|
|          New Jersey|             350.8|
|            Colorado|333.46666666666664|
|          Washington|266.29999999999995|
|       Massachusetts|             261.8|
|         Connecticut|250.13333333333333|
|                Utah|             243.3|
|       New Hampshire|             237.4|
|              Nevada|             237.3|
|        Rhode Island|             233.3|
|              Oregon|231.53333333333333|
|            Maryland|             223.1|
|           Wisconsin|             210.8|
|           Minnesota|             209.4|
|            Virginia|            201.25|
|             Arizona|             192.1|
|            New York|           185.775|
|        Pennsylvania|             183.8|
+--------------------+------------

Filter cities with population > 1 million 

In [0]:

display(df.select("City","2014 Population estimate").filter(df["2014 Population estimate"]>1000000).orderBy("2014 Population estimate"))

City,2014 Population estimate
San Jose,1015785
Dallas,1281047
San Diego,1381069
San Antonio,1436697
Phoenix,1537058
Philadelphia[8],1560297
Houston[7],2239558
Chicago,2722389
Los Angeles,3928864
New York[6],8491079


Databricks visualization. Run in Databricks to view.

Identify cities with missing or zero price values

In [0]:
df.filter(df["2015 median sales price"].isNull() | df["2015 median sales price"]==0).show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1077165434867399>:1
----> 1 df.filter(df["2015 median sales price"].isNull() | df["2015 median sales price"]==0).show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3125, in DataFrame.filter(self, condition)
   3123     jdf = self._jdf.filter(condition)
   3124 elif isinstance(condition, Column):
-> 3125     jdf = self._jdf.filter(condition._jc)
   3126 else:
   3127     raise PySparkTypeError(
   3128         error_class="NOT_COLUMN_OR_STRING",
